# <center> pyspark presentation 
    <center>

# $\color{red}{\text{Tasks for first week:}}$

 ## 1-calculate distance of each trip using haversine library and add the result to the dataset
 ## 2-calculate the duration in seconds of each trip
 ## 3-by assuming each minute cost 0.35 cent calculate the fee for each trip

# $\color{red}{\text{Tasks for second week :}}$
 ## 1-calculate the total distance for each bike and list the top 10
 ## 2-calculate the number of trips for each start station list top 10 and find the ratio of using as male or female
 ## 3-make a comparison to find the percentage of usage for customer and subscriber
 ## 4-calculate the age of all users and show the relation between the distance and the age

# $\color{red}{\text{Tasks for third week :}}$ 
 ## 1-calculate the total cost for all customers and all subscribers
 ## 2- what is the ratio of payment using cc or app wallet
 ## 3-what is the preferred way to pay for customers and subscriber

Ideas for presentaion :



 ## 1-calculate distance of each trip using haversine library and add the result to the dataset

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sqrt
spark = SparkSession.builder.appName("MyApp").getOrCreate()
df = spark.read.csv('Downloads/2017-fordgobike-tripdata.csv', header=True, inferSchema=True)


In [2]:
#from pyspark.sql.functions import acos, cos, sin, radians, least
#from pyspark.sql.functions import acos, cos, sin, sqrt, lit
from pyspark.sql.functions import sin, cos, sqrt, asin, radians, least, acos, col, lit


def haversine(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = radians(lat1), radians(long1), radians(lat2), radians(long2)
    dlat = lat2 - lat1
    dlon = long2 - long1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    m = 6367 * c * 1000
    return m


# Read the dataset into a Spark DataFrame
df = spark.read.csv('Downloads/2017-fordgobike-tripdata.csv', header=True, inferSchema=True)

# Calculate the distance for each trip using the Haversine function
df = df.withColumn('distance_m', haversine('start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude'))

# Show the first 10 rows of the updated DataFrame
df.show(10)


+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|        distance_m|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+
|   57:39.7| 12:50.2|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96|  C

In [3]:
df.select('distance_m').show(10)

+------------------+
|        distance_m|
+------------------+
| 942.3373818591812|
|3067.7986569373347|
|               0.0|
|1045.9655184811572|
| 635.9398610767416|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
+------------------+
only showing top 10 rows



In [4]:
from pyspark.sql.functions import percentile_approx

# Get the median trip distance
median_distance = df.selectExpr("percentile_approx(distance_m, 0.5)").collect()[0][0]

# Print the result
print("Median distance: {:.2f} meters".format(median_distance))

Median distance: 1399.36 meters


 ## 2-calculate the duration in seconds of each trip


In [5]:
from pyspark.sql.functions import expr, when, col

df = df.withColumn('start_time', expr("to_timestamp(start_time, 'mm:ss.S')"))
df = df.withColumn('end_time', expr("to_timestamp(end_time, 'mm:ss.S')"))

df = df.withColumn('duration_sec',
                   when(col('end_time') < col('start_time'),
                        (col('end_time').cast('long') + 3600 - col('start_time').cast('long')))
                   .otherwise(col('end_time').cast('long') - col('start_time').cast('long')))


In [6]:
df.printSchema() # _c0 is unneccery column

root
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_m: double (nullable = true)
 |-- duration_sec: long (nullable = true)



In [8]:
df.select(col('bike_id'),('duration_sec')).show(10)

+-------+------------+
|bike_id|duration_sec|
+-------+------------+
|     96|         911|
|     88|        3201|
|   1094|        2568|
|   2831|         973|
|   3167|         403|
|   1487|        2027|
|   3539|         907|
|   1503|         734|
|   3125|         551|
|   2543|         639|
+-------+------------+
only showing top 10 rows



In [9]:
from pyspark.sql.functions import avg

# Calculate the average trip duration in seconds
avg_duration = df.agg(avg("duration_sec")).collect()[0][0]

# Print the result
print("The average trip duration is {:.2f} seconds".format(avg_duration))


The average trip duration is 738.29 seconds


## 3-by assuming each minute cost 0.35 cent calculate the fee for each trip

In [10]:
from pyspark.sql.functions import round

df = df.withColumn('fee', round(df.duration_sec/60 * 0.35, 2))

In [14]:
df_fee = df.select(col('bike_id'),('fee')).show(10)
df_fee

+-------+-----+
|bike_id|  fee|
+-------+-----+
|     96| 5.31|
|     88|18.67|
|   1094|14.98|
|   2831| 5.68|
|   3167| 2.35|
|   1487|11.82|
|   3539| 5.29|
|   1503| 4.28|
|   3125| 3.21|
|   2543| 3.73|
+-------+-----+
only showing top 10 rows



## 4-calculate the total distance for each bike and list the top 10

In [16]:
# Group the data by bike_id and calculate the total distance for each bike
grouped_data = df.groupBy("bike_id").agg({"distance_m": "sum"})

# Rename the column from sum(Distance_in_meter) to total_distance
grouped_data = grouped_data.withColumnRenamed("sum(distance_m)", "total_distance")

# Round the total_distance column to 2 decimal places
grouped_data = grouped_data.withColumn("total_distance", round("total_distance", 2))

# Sort the data by total_distance in descending order and select the top 10
top_10 = grouped_data.sort(grouped_data.total_distance.desc()).limit(10)

# Show the results
top_10.show()

+-------+--------------+
|bike_id|total_distance|
+-------+--------------+
|     68|     742901.74|
|   2178|     720728.46|
|    256|     671493.31|
|    235|     669740.32|
|   2049|     656414.81|
|    441|      656229.1|
|   2226|      647415.6|
|    796|      646460.7|
|    190|     639891.29|
|   2365|     639010.34|
+-------+--------------+



## 5-calculate the number of trips for each start station list top 10 and find the ratio of using as male or female

In [17]:
from pyspark.sql.functions import count, sum
from pyspark.sql.window import Window

# Group the data by start station and count the number of trips
grouped_data = df.groupBy("start_station_name").agg(count("start_station_name").alias("number_of_trips"))

# Sort the data by number_of_trips in descending order and select the top 10
top_10 = grouped_data.sort(grouped_data.number_of_trips.desc()).limit(10)

# Show the results
top_10.show()

+--------------------+---------------+
|  start_station_name|number_of_trips|
+--------------------+---------------+
|San Francisco Fer...|          15187|
|The Embarcadero a...|          13664|
|San Francisco Cal...|          12546|
|San Francisco Cal...|          12055|
|Market St at 10th St|          11960|
|Montgomery St BAR...|          11334|
|  Berry St at 4th St|          10956|
|Powell St BART St...|          10142|
|Howard St at Beal...|           9926|
|Steuart St at Mar...|           9347|
+--------------------+---------------+



In [18]:
from pyspark.sql.functions import count, sum, col, round
from pyspark.sql.window import Window

# Group the data by member_gender and calculate the count for each gender
gender_count = df.groupBy("member_gender").agg(count("member_gender").alias("gender_count"))

# Calculate the total count of all genders
total_count = gender_count.agg(sum("gender_count")).collect()[0][0]

# Calculate the ratio of each gender and round to 2 decimal places
gender_ratio = gender_count.withColumn("ratio", round(col("gender_count") / total_count, 2))

# Show the results
gender_ratio.show()



+-------------+------------+-----+
|member_gender|gender_count|ratio|
+-------------+------------+-----+
|         null|           0|  0.0|
|       Female|       98621| 0.22|
|        Other|        6299| 0.01|
|         Male|      348318| 0.77|
+-------------+------------+-----+



 ## 6-make a comparison to find the percentage of usage for customer and subscriber


In [19]:
percent_user = df.groupBy('user_type').agg(count('*').alias('total'))

percent_user.withColumn('percantage',round(col('total')/ df.count()*100)).show()

+----------+------+----------+
| user_type| total|percantage|
+----------+------+----------+
|Subscriber|409230|      79.0|
|  Customer|110470|      21.0|
+----------+------+----------+



 ## 7-calculate the age of all users and show the relation between the distance and the age

In [34]:
# calculate the age of each user

from pyspark.sql import functions as F

df1 = df.withColumn("age", 2022 - F.col("member_birth_year"))


In [35]:
df1.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_m: double (nullable = true)
 |-- duration_sec: long (nullable = true)
 |-- fee: double (nullable = true)
 |-- age: integer (nullable = true)



In [38]:
df_age = df1.select(col('age'),('distance_m')).show(10)
df_age

+----+------------------+
| age|        distance_m|
+----+------------------+
|  35| 942.3373818591812|
|  57|3067.7986569373347|
|null|               0.0|
|null|1045.9655184811572|
|  25| 635.9398610767416|
|null|               0.0|
|  31|               0.0|
|null|               0.0|
|null|               0.0|
|null|               0.0|
+----+------------------+
only showing top 10 rows



In [36]:
#find the correlation between age and total distance

from pyspark.sql.functions import *
df1.stat.corr("age","distance_m")

0.011148261513315322

In [39]:

# Show the relation between the distance and the age
df1.groupBy("age").agg({"distance_m": "mean"}).show()


+----+------------------+
| age|   avg(distance_m)|
+----+------------------+
|  31|1627.1747321491512|
|  65|1550.7970469464924|
|  53|1615.9961666112085|
|  78| 1738.533853449056|
|  34| 1632.586214667949|
| 115|1258.8515568959915|
|  81|1545.7800030706612|
|  28|1597.8047652824787|
|  76| 978.6525959353298|
|  27|1614.1143983222787|
|  26| 1268.911708273193|
|  44| 1586.594792391467|
| 122|1423.9429793322101|
| 111|1891.3312419369552|
|  47|1590.3443459610605|
|null| 1502.649140519337|
|  52|1680.2273796697316|
|  40|1614.2863442246678|
|  94|1258.9165538476107|
|  57| 1530.291219745407|
+----+------------------+
only showing top 20 rows



 ## 8-calculate the total cost for all customers and all subscribers

In [ ]:
df = df.withColumn('total_paid', 
                   round((df.distance_m/ 1000) * .35)) # assume 35 cent for every meter


df.groupBy("user_type").agg(sum('total_paid')
                            .alias('total_paid_sum')).show()

 ## 9- what is the ratio of payment using cc or app wallet

In [ ]:
# what is the ratio of payment using cc or app wallet

df.groupBy('pyment').agg(count('*').alias('number of usage')).show()

 ## 10-what is the preferred way to pay for customers and subscriber

In [ ]:
df.groupBy('pyment').agg(count('*').alias('total_on_type'))\
.withColumn('perc', col('total_on_type') / df.count()).show()

In [ ]:
df.groupBy('user_type', 'pyment').agg(count('*').alias('row_count')).withColumnRenamed('pyment', 'payment').show()

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

import numpy as np

path = r'E:\Learning Pandas\Data_Manupilation\2017-fordgobike-tripdata.csv'

df1 = pd.read_csv(path)

df1.sample(5)
df1.member_birth_year.dtype